
# 1 - Imports and defining functions

In [ ]:
import os
import numpy as np

from pyFM.mesh import TriMesh
from tqdm.auto import tqdm

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# 2 - Loading Data

A functional map network expects two objects as input:
 - A list `meshlist` of all the meshes in the networks (nodes in the graph)
 - A dictionary `maps_dict` containing functional maps, where keys are of the shape `(i,j)` with `i` and `j` the indices of the meshes in `meshlist`


We have access both to the meshes and to noisy *pointwise maps*, which we will have to convert to functional maps !

Let's first load the meshes

In [ ]:
meshlist = [TriMesh(f'../data/camel_gallop/camel-gallop-{i:02d}.off', area_normalize=True, center=True).process(k=150, intrinsic=True) for i in tqdm(range(1,11))]

In [ ]:
double_plot(meshlist[0], meshlist[5])

Let's now load the pointwise maps and convert them to functional maps using pyFM, thus building `maps_dict`

In [ ]:
import pyFM.spectral as spectral

K = 30  # Size of initial functional maps, small value since our initial maps have noise

# All pointwise maps are located here, with format 'ind2_to_ind1' for the map from mesh ind2 to mesh ind1
map_files = os.listdir('../data/camel_gallop/maps')

maps_dict = {}

for map_filename in tqdm(map_files):
    ind2, ind1 = map_filename.split('_to_')
    ind1, ind2 = int(ind1), int(ind2)

    # Indicing starts at 1 in the names, but at 0 on the meshlist
    mesh1, mesh2 = meshlist[ind1-1], meshlist[ind2-1]
    
    # Load the pointwise map
    p2p_21 = np.loadtxt(f'../data/camel_gallop/maps/{map_filename}', dtype=int)

    # Convert to functional map
    FM_12 = spectral.mesh_p2p_to_FM(p2p_21, mesh1, mesh2, dims=K)
    
    # Populate the dictionary
    maps_dict[(ind1-1, ind2-1)] = FM_12

# 3 - Building the Canonical Consistent Latent Basis

Let's build the FMN model and the canonical latent basis

In [ ]:
from pyFM.FMN import FMN

In [ ]:
# Build the network
fmn_model = FMN(meshlist, maps_dict.copy())

# Compute CCLB
fmn_model.compute_CCLB(m=20)

# 4 - Analyze the embedding

We can use the Characterisic Shape Differences as embedding of each shape.
We have access to both the *area* and *conformal* characteristic shape difference operator for each shape using the CCLB.

In [ ]:
all_embs_a = []
all_embs_c = []

for i in range(fmn_model.n_meshes):
    CSD_a, CSD_c = fmn_model.get_CSD(i)

    all_embs_a.append(CSD_a.flatten())
    all_embs_c.append(CSD_c.flatten())

all_embs_a = np.array(all_embs_a)
all_embs_c = np.array(all_embs_c)

Let's apply PCA on the embedding to visualize the results

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
pca_model = PCA(n_components=2)
emb_red_a = pca_model.fit_transform(all_embs_a)
emb_red_c = pca_model.fit_transform(all_embs_c)

In [ ]:
_, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].scatter(emb_red_a[:, 0], emb_red_a[:, 1], c=np.arange(len(emb_red_a)))
axs[0].set_title('Area CSD')

axs[1].scatter(emb_red_c[:, 0], emb_red_c[:, 1], c=np.arange(len(emb_red_c)))
axs[1].set_title('Conformal CSD')

# 5 - Apply Consistent ZoomOut

In [ ]:
fmn_model.zoomout_refine(nit=10, step=5, subsample=3000, isometric=True, weight_type='icsm',
                    M_init=None, cclb_ratio=.9, n_jobs=1, equals_id=False,
                    verbose=True)

In [ ]:
fmn_model.compute_CCLB(m=int(0.9*fmn_model.M))

In [ ]:
fmn_model.cclb_eigenvalues.shape

In [ ]:
all_embs_a = []
all_embs_c = []

for i in range(fmn_model.n_meshes):
    CSD_a, CSD_c = fmn_model.get_CSD(i)

    all_embs_a.append(CSD_a.flatten())
    all_embs_c.append(CSD_c.flatten())

all_embs_a = np.array(all_embs_a)
all_embs_c = np.array(all_embs_c)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
pca_model = PCA(n_components=2)
emb_red_a = pca_model.fit_transform(all_embs_a)
emb_red_c = pca_model.fit_transform(all_embs_c)

In [ ]:
_, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].scatter(emb_red_a[:, 0], emb_red_a[:, 1], c=np.arange(len(emb_red_a)))
axs[0].set_title('Area CSD')

axs[1].scatter(emb_red_c[:, 0], emb_red_c[:, 1], c=np.arange(len(emb_red_c)))
axs[1].set_title('Conformal CSD')